<a href="https://colab.research.google.com/github/Reusezer/Neuron-activations/blob/main/Qwen_hooks_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1-1. upgrade pip (optional)
!pip install -q --upgrade pip

# 1-2. core libs: HF Transformers, accelerate (for device mapping), pandas, datasets, torch
!pip install -q transformers accelerate pandas datasets torch


In [ ]:
import os
import torch
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM

# choose GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


In [ ]:
MODEL_NAME = "Qwen/Qwen3-8B"

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# HF model
hf_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)
hf_model.eval()


In [ ]:
CSV_URL = "https://raw.githubusercontent.com/nyu-mll/crows-pairs/master/data/crows_pairs_anonymized.csv"
csv_path = "crows_pairs_anonymized.csv"

if not os.path.exists(csv_path):
    import urllib.request
    urllib.request.urlretrieve(CSV_URL, csv_path)

df = pd.read_csv(csv_path).dropna(subset=["sent_more","sent_less"])
print("Total pairs loaded:", len(df))
df.head()


In [ ]:
row = df.sample(1).iloc[0]
sent_more, sent_less = row["sent_more"], row["sent_less"]

print("MORE (biased):", sent_more)
print("LESS (neutral):", sent_less)


In [ ]:
# List all submodules whose name contains "mlp" and are Linear layers:
for name, module in hf_model.named_modules():
    if "mlp" in name.lower() and isinstance(module, torch.nn.Linear):
        print(f"{name:60s} — weight shape: {tuple(module.weight.shape)}")


In [ ]:
# Cell A: Define known-working down_proj layers for Qwen3-8B (layers 27–35)
HOOKED_MODULES = [
    f"model.layers.{i}.mlp.down_proj" for i in range(27, 36)
]
print("Hooking modules:")
for m in HOOKED_MODULES:
    print("  ", m)




In [ ]:
from collections import defaultdict
import torch

def get_activations(prompt: str):
    activations = defaultdict(list)

    def make_hook(name):
        def hook(module, input, output):
            activations[name].append(output.detach().cpu())
        return hook

    # Register hooks on each relevant layer
    hooks = []
    for name, module in hf_model.named_modules():
        if name in HOOKED_MODULES:
            hooks.append(module.register_forward_hook(make_hook(name)))

    # Forward pass
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        hf_model(**inputs)

    for h in hooks:
        h.remove()

    # Flatten hook outputs
    return {
        name: torch.cat(acts, dim=0) for name, acts in activations.items()
    }



In [ ]:
import torch
from tqdm import tqdm
from google.colab import files

records = []

# ✅ Loop through every CrowS-Pairs row
for _, row in tqdm(df.iterrows(), total=len(df), desc="Comparing CrowS-Pairs"):
    try:
        sent_more = row["sent_more"]
        sent_less = row["sent_less"]

        act_more = get_activations(sent_more)
        act_less = get_activations(sent_less)

        for name in HOOKED_MODULES:
            if name in act_more and name in act_less:
                min_len = min(act_more[name].shape[1], act_less[name].shape[1])
                delta = (act_more[name][:, :min_len, :] - act_less[name][:, :min_len, :]).abs().mean(dim=(0, 1))

                topk = torch.topk(delta, 5)

                for idx, val in zip(topk.indices.tolist(), topk.values.tolist()):
                    records.append({
                        "module": name,
                        "layer": int(name.split(".")[2]),
                        "neuron_index": idx,
                        "delta_value": float(val),
                        "prompt_more": sent_more,
                        "prompt_less": sent_less
                    })
    except Exception as e:
        print(f"❗ Error on row: {e}")

# ✅ Save to CSV
if records:
    df_out = pd.DataFrame(records)
    output_path = "crows_bias_top_neurons.csv"
    df_out.to_csv(output_path, index=False)
    print(f"✅ Saved {len(df_out)} rows to {output_path}")
    files.download(output_path)
else:
    print("⚠️ No neuron activations recorded.")





In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from google.colab import files

records = []

# Sample just 10 examples for now (fast test)
sampled_df = df.sample(10, random_state=42)

for _, row in tqdm(sampled_df.iterrows(), total=len(sampled_df), desc="Comparing Pairs"):
    try:
        more, less = row["sent_more"], row["sent_less"]
        act_more = get_activations(more)
        act_less = get_activations(less)

        for name in HOOKED_MODULES:
            if name in act_more and name in act_less:
                delta = (act_more[name] - act_less[name]).abs().mean(dim=(0, 1))
                topk = torch.topk(delta, 5)
                for idx, val in zip(topk.indices.tolist(), topk.values.tolist()):
                    records.append({
                        "module": name,
                        "layer": int(name.split(".")[2]),
                        "neuron_index": idx,
                        "delta_value": float(val),
                        "prompt_more": more,
                        "prompt_less": less
                    })
    except Exception as e:
        print(f"❗ Error with pair: {e}")

# Save if any records
if records:
    df_out = pd.DataFrame(records)
    out_path = "crows_bias_top_neurons.csv"
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved {len(df_out)} rows to {out_path}")
    files.download(out_path)
else:
    print("⚠️ No neuron deltas recorded. Check hook layer matches.")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load your updated CSV
df = pd.read_csv("/content/crows_bias_top_neurons (1).csv")

# Aggregate: mean delta per neuron per layer
df["layer"] = df["module"].str.extract(r"layers\.(\d+)")
df["layer"] = df["layer"].astype(int)

pivot = df.pivot_table(
    index="layer",
    columns="neuron_index",
    values="delta_value",
    aggfunc="mean"
).fillna(0)

# Plot heatmap
plt.figure(figsize=(14, 6))
sns.heatmap(pivot, cmap="viridis", xticklabels=False)
plt.title("Bias-Activated Neurons by Layer (mean ∆ activation)")
plt.xlabel("Neuron Index")
plt.ylabel("Layer")
plt.tight_layout()
plt.show()

